In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, r2_score

In [2]:
df = pd.read_csv('train_dataset_train.csv')
df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


In [3]:
# просмотр повторяющихся строк

duplicateRows = df[df.duplicated()]
duplicateRows

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label


In [4]:
# удалим ненужные столбцы

df = df.drop(['id', 'entrance_nm', 'ticket_id', 'entrance_id', 'station_id', 'line_id'], axis=1)

In [5]:
df

,ticket_type_nm,station_nm,line_nm,pass_dttm,time_to_under,label
0,Пропуск FacePay,Лефортово,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,СК учащегося 30 дней,Войковская,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,БСК дружинника г.Москвы,Войковская,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,30 дней,Войковская,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,КОШЕЛЕК,Войковская,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...
1091016,Пропуск FacePay,Нижегородская,Некрасовская,2022-09-19 02:54:50,124.750000,15005
1091017,Пропуск FacePay,Калужская,Калужско-Рижская,2022-09-19 03:06:02,56.066667,6007
1091018,Пропуск FacePay,Каширская,Замоскворецкая,2022-09-19 03:17:00,91.283333,2010
1091019,Пропуск FacePay,Воронцовская,Большая кольцевая,2022-09-19 03:23:30,47.900000,11014


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091021 entries, 0 to 1091020
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   ticket_type_nm  1091021 non-null  object 
 1   station_nm      1091021 non-null  object 
 2   line_nm         1091021 non-null  object 
 3   pass_dttm       1091021 non-null  object 
 4   time_to_under   1091021 non-null  float64
 5   label           1091021 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 49.9+ MB


In [7]:
values = ((df.isna().sum() / len(df)) * 100).sort_values()
count = 0

for i in values:
    if i == 0:
        count += 1
print(f'Количество полностью заполненных объектов - {count}')
print(f'Их процент из всей выборки - {int(count / len (values) * 100)}%')

Количество полностью заполненных объектов - 6
Их процент из всей выборки - 100%


Распределим типы проездных по категориям

In [8]:
df.ticket_type_nm.unique()

array(['Пропуск FacePay', 'СК учащегося 30 дней',
       'БСК дружинника г.Москвы', '30 дней', 'КОШЕЛЕК',
       'СК студента 30 дней', '90 дней', '90 дней ЕДИНЫЙ ТК',
       '30 дней СК учащегося', '90 дней СК студента',
       '30 дней СК студента', 'ББК', 'СК аспиранта 90 дней',
       'Социальная карта москвича', '30 дней ЕДИНЫЙ ТК',
       'Социальная карта жителя Моск. области', '30 дней Пригород ТК',
       'Социальная карта москвича с сопровождающим',
       'Пропуск сотрудника УВД по охране Метрополитена',
       '60 поездок ЕДИНЫЙ ТК', 'СК для сотрудника ГУВД г.Москвы',
       '365 дней ЕДИНЫЙ ТК', 'ВЕСБ МОСКВА (7мин)', 'СК студента 90 дней',
       '90 дней Пригород ТК', '30 дней  Пригород', '365 дней Пригород ТК',
       '365 дней', 'СК ординатора 30 дней', '30 дней СК аспиранта',
       '30 дней СК ординатора', '90 дней СК ординатора',
       'ВЕСБ МО (7 мин)', 'ВЕСБ МО (тип 5)', 'СК ординатора 90 дней',
       'СК аспиранта 30 дней', '90 дней СК учащегося',
       'Времен

In [9]:
students = ['СК студента 30 дней', '90 дней СК студента', 'СК учащегося 30 дней', '30 дней СК учащегося', 
            'СК аспиранта 90 дней', 'СК студента 90 дней', 'СК ординатора 30 дней', '30 дней СК аспиранта',
            '30 дней СК ординатора', '90 дней СК ординатора', 'СК ординатора 90 дней', 'СК аспиранта 30 дней', 
            '90 дней СК учащегося', 'СК учащегося 90 дней', '90 дней СК аспиранта', 'СК ассистента-стажера 30 дней', 
            'СК ассистента-стажера 90 дней', '30 дней СК ассистента-стажера', '30 дней СК студента']

In [10]:
df.loc[df['ticket_type_nm'].isin(students), 'ticket_type_nm'] = 'stud'

In [11]:
df.ticket_type_nm.unique()

array(['Пропуск FacePay', 'stud', 'БСК дружинника г.Москвы', '30 дней',
       'КОШЕЛЕК', '90 дней', '90 дней ЕДИНЫЙ ТК', 'ББК',
       'Социальная карта москвича', '30 дней ЕДИНЫЙ ТК',
       'Социальная карта жителя Моск. области', '30 дней Пригород ТК',
       'Социальная карта москвича с сопровождающим',
       'Пропуск сотрудника УВД по охране Метрополитена',
       '60 поездок ЕДИНЫЙ ТК', 'СК для сотрудника ГУВД г.Москвы',
       '365 дней ЕДИНЫЙ ТК', 'ВЕСБ МОСКВА (7мин)', '90 дней Пригород ТК',
       '30 дней  Пригород', '365 дней Пригород ТК', '365 дней',
       'ВЕСБ МО (7 мин)', 'ВЕСБ МО (тип 5)', 'Временный билет ММ',
       'Социальная карта жителя Моск. области с сопровождающим',
       '3 суток Единый ТК', 'Пропуск сотрудника  метрополитена',
       'ВЕСБ МОСКВА', 'Безналичная транспортная карта',
       '90 дней  Пригород', '1 сутки ЕДИНЫЙ ТК', 'Ультралайт Единый (70)',
       'Пропуск руководителя метрополитена', 'Пропуск  сотрудника МЦК',
       '365 дней  Пригород', 

In [12]:
guests = ['Билет 1 сутки ЕДИНЫЙ', 'Билет 3 суток ЕДИНЫЙ', '1 сутки ЕДИНЫЙ ТК', '3 суток Единый ТК', '1 сутки Пригород ТК',
          'ВЕСБ МОСКВА (7мин)', 'ВЕСБ МО (7 мин)', 'Временный билет ММ']

In [13]:
df.loc[df['ticket_type_nm'].isin(guests), 'ticket_type_nm'] = 'guest'

In [14]:
df.ticket_type_nm.unique()

array(['Пропуск FacePay', 'stud', 'БСК дружинника г.Москвы', '30 дней',
       'КОШЕЛЕК', '90 дней', '90 дней ЕДИНЫЙ ТК', 'ББК',
       'Социальная карта москвича', '30 дней ЕДИНЫЙ ТК',
       'Социальная карта жителя Моск. области', '30 дней Пригород ТК',
       'Социальная карта москвича с сопровождающим',
       'Пропуск сотрудника УВД по охране Метрополитена',
       '60 поездок ЕДИНЫЙ ТК', 'СК для сотрудника ГУВД г.Москвы',
       '365 дней ЕДИНЫЙ ТК', 'guest', '90 дней Пригород ТК',
       '30 дней  Пригород', '365 дней Пригород ТК', '365 дней',
       'ВЕСБ МО (тип 5)',
       'Социальная карта жителя Моск. области с сопровождающим',
       'Пропуск сотрудника  метрополитена', 'ВЕСБ МОСКВА',
       'Безналичная транспортная карта', '90 дней  Пригород',
       'Ультралайт Единый (70)', 'Пропуск руководителя метрополитена',
       'Пропуск  сотрудника МЦК', '365 дней  Пригород', 'ВЛБ МОСКВА',
       'ВЕСБ МО с сопровождающим', 'ВЕСБ МО (тип 6)'], dtype=object)

In [15]:
work = ['БСК дружинника г.Москвы', '30 дней', '90 дней', '90 дней ЕДИНЫЙ ТК', '30 дней ЕДИНЫЙ ТК',
        '30 дней Пригород ТК', '60 поездок ЕДИНЫЙ ТК', 
        'СК для сотрудника ГУВД г.Москвы', '365 дней ЕДИНЫЙ ТК', '90 дней Пригород ТК', '30 дней  Пригород',
        '365 дней Пригород ТК', '365 дней', '90 дней  Пригород', '365 дней  Пригород', 'КОШЕЛЕК',
        'Безналичная транспортная карта', 'Пропуск FacePay', 'Ультралайт Единый (70)', 'ББК']

In [16]:
df.loc[df['ticket_type_nm'].isin(work), 'ticket_type_nm'] = 'work'

In [17]:
df.ticket_type_nm.unique()

array(['work', 'stud', 'Социальная карта москвича',
       'Социальная карта жителя Моск. области',
       'Социальная карта москвича с сопровождающим',
       'Пропуск сотрудника УВД по охране Метрополитена', 'guest',
       'ВЕСБ МО (тип 5)',
       'Социальная карта жителя Моск. области с сопровождающим',
       'Пропуск сотрудника  метрополитена', 'ВЕСБ МОСКВА',
       'Пропуск руководителя метрополитена', 'Пропуск  сотрудника МЦК',
       'ВЛБ МОСКВА', 'ВЕСБ МО с сопровождающим', 'ВЕСБ МО (тип 6)'],
      dtype=object)

In [18]:
metro = ['Пропуск сотрудника УВД по охране Метрополитена', 'Пропуск сотрудника  метрополитена',
        'Пропуск руководителя метрополитена', 'Пропуск  сотрудника МЦК']

In [19]:
df.loc[df['ticket_type_nm'].isin(metro), 'ticket_type_nm'] = 'metro'

In [20]:
df.ticket_type_nm.unique()

array(['work', 'stud', 'Социальная карта москвича',
       'Социальная карта жителя Моск. области',
       'Социальная карта москвича с сопровождающим', 'metro', 'guest',
       'ВЕСБ МО (тип 5)',
       'Социальная карта жителя Моск. области с сопровождающим',
       'ВЕСБ МОСКВА', 'ВЛБ МОСКВА', 'ВЕСБ МО с сопровождающим',
       'ВЕСБ МО (тип 6)'], dtype=object)

In [21]:
# ВЕСБ - временные единые социальные билеты
# ВЛБ - временные льготные билеты

lgot = ['Социальная карта москвича с сопровождающим', 'Социальная карта жителя Моск. области с сопровождающим',
        'ВЕСБ МО с сопровождающим', 'Социальная карта жителя Моск. области', 'Социальная карта москвича', 'ВЕСБ МО (тип 5)',
        'ВЕСБ МОСКВА', 'ВЕСБ МО (тип 6)', 'ВЛБ МОСКВА']

In [22]:
df.loc[df['ticket_type_nm'].isin(lgot), 'ticket_type_nm'] = 'lg'

In [23]:
ticket_types = df.ticket_type_nm.unique()
ticket_types

array(['work', 'stud', 'lg', 'metro', 'guest'], dtype=object)

In [24]:
df

,ticket_type_nm,station_nm,line_nm,pass_dttm,time_to_under,label
0,work,Лефортово,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,stud,Войковская,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,work,Войковская,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,work,Войковская,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,work,Войковская,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...
1091016,work,Нижегородская,Некрасовская,2022-09-19 02:54:50,124.750000,15005
1091017,work,Калужская,Калужско-Рижская,2022-09-19 03:06:02,56.066667,6007
1091018,work,Каширская,Замоскворецкая,2022-09-19 03:17:00,91.283333,2010
1091019,work,Воронцовская,Большая кольцевая,2022-09-19 03:23:30,47.900000,11014


Поработаем с датами

In [25]:
def to_datetime(val):
    dateString = val
    dateFormatter = "%Y-%m-%d %H:%M:%S"
    val = datetime.strptime(dateString, dateFormatter)
    return val.day, val.hour 

In [26]:
# нас интересуют только день и час, в который была валидация

new_col_day = []
new_col_hour = []
for i in range(len(df['pass_dttm'])):
    day, hour = to_datetime(df['pass_dttm'][i])
    new_col_day.append(day)
    new_col_hour.append(hour)
df['day'] = new_col_day
df['hour'] = new_col_hour

In [27]:
df = df.drop(['pass_dttm'], axis=1)

In [28]:
df

,ticket_type_nm,station_nm,line_nm,time_to_under,label,day,hour
0,work,Лефортово,Большая кольцевая,216.316667,8001,12,5
1,stud,Войковская,Замоскворецкая,648.183333,9011,12,5
2,work,Войковская,Замоскворецкая,865.333333,7022,12,5
3,work,Войковская,Замоскворецкая,1048.233333,2022,12,5
4,work,Войковская,Замоскворецкая,965.600000,2017,12,5
...,...,...,...,...,...,...,...
1091016,work,Нижегородская,Некрасовская,124.750000,15005,19,2
1091017,work,Калужская,Калужско-Рижская,56.066667,6007,19,3
1091018,work,Каширская,Замоскворецкая,91.283333,2010,19,3
1091019,work,Воронцовская,Большая кольцевая,47.900000,11014,19,3


In [29]:
df.day.unique()

array([12, 13, 14, 15, 16, 17, 18, 19], dtype=int64)

In [30]:
# 17 и 18 сентября - выходные

df['weekend'] = np.where(df['day'].isin([17, 18]), 'yes', 'no')

In [31]:
df

,ticket_type_nm,station_nm,line_nm,time_to_under,label,day,hour,weekend
0,work,Лефортово,Большая кольцевая,216.316667,8001,12,5,no
1,stud,Войковская,Замоскворецкая,648.183333,9011,12,5,no
2,work,Войковская,Замоскворецкая,865.333333,7022,12,5,no
3,work,Войковская,Замоскворецкая,1048.233333,2022,12,5,no
4,work,Войковская,Замоскворецкая,965.600000,2017,12,5,no
...,...,...,...,...,...,...,...,...
1091016,work,Нижегородская,Некрасовская,124.750000,15005,19,2,no
1091017,work,Калужская,Калужско-Рижская,56.066667,6007,19,3,no
1091018,work,Каширская,Замоскворецкая,91.283333,2010,19,3,no
1091019,work,Воронцовская,Большая кольцевая,47.900000,11014,19,3,no


In [32]:
df.hour.unique()

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23,  0,  1,  2,  3,  4], dtype=int64)

In [33]:
def rule(x):
    if x>=6 and x<10:
        return 'to_work'
    elif x>=10 and x<17:
        return 'tasks'
    elif x>=17 and x<22:
        return 'to_home'
    else:
        return 'party'

In [34]:
# в зависимости от времени валидации можно предположить маршрут человека

df['goal'] = df.apply(lambda x: rule(x['hour']), axis =  1)

In [35]:
df = df.drop(['day', 'hour'], axis=1)

In [36]:
df

,ticket_type_nm,station_nm,line_nm,time_to_under,label,weekend,goal
0,work,Лефортово,Большая кольцевая,216.316667,8001,no,party
1,stud,Войковская,Замоскворецкая,648.183333,9011,no,party
2,work,Войковская,Замоскворецкая,865.333333,7022,no,party
3,work,Войковская,Замоскворецкая,1048.233333,2022,no,party
4,work,Войковская,Замоскворецкая,965.600000,2017,no,party
...,...,...,...,...,...,...,...
1091016,work,Нижегородская,Некрасовская,124.750000,15005,no,party
1091017,work,Калужская,Калужско-Рижская,56.066667,6007,no,party
1091018,work,Каширская,Замоскворецкая,91.283333,2010,no,party
1091019,work,Воронцовская,Большая кольцевая,47.900000,11014,no,party


Закодируем строковые признаки

In [37]:
h_ticket_type = FeatureHasher(n_features=5, input_type='string')
hashed_Feature = h_ticket_type.fit_transform(df['ticket_type_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_station = FeatureHasher(n_features=20, input_type='string')
hashed_Feature = h_station.fit_transform(df['station_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_line = FeatureHasher(n_features=10, input_type='string')
hashed_Feature = h_line.fit_transform(df['line_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_goal = FeatureHasher(n_features=5, input_type='string')
hashed_Feature = h_ticket_type.fit_transform(df['goal'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_weekend = FeatureHasher(n_features=5, input_type='string')
hashed_Feature = h_ticket_type.fit_transform(df['weekend'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

In [38]:
df = df.drop(['ticket_type_nm', 'station_nm', 'line_nm', 'goal', 'weekend'], axis=1)

In [39]:
df

,time_to_under,label,0,1,2,3,4,0,1,2,...,0,1,2,3,4,0,1,2,3,4
0,216.316667,8001,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1,648.183333,9011,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
2,865.333333,7022,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
3,1048.233333,2022,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
4,965.600000,2017,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091016,124.750000,15005,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1091017,56.066667,6007,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1091018,91.283333,2010,1.0,-2.0,0.0,1.0,0.0,0.0,-1.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1091019,47.900000,11014,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0


In [40]:
X = df.drop(['time_to_under', 'label'], axis=1)
y = df['time_to_under']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = LinearRegression()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

In [41]:
r2 = r2_score(y_test, y_predict)
r2

0.4321204666845829

In [42]:
X = df.drop(['time_to_under', 'label'], axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestClassifier(n_estimators=10)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

In [43]:
recall = recall_score(y_test, y_predict, average='micro')
recall

0.06515900973703587

In [44]:
result = 0.5 * r2 + 0.5 * recall
print('result: ' + str(result))

result: 0.2486397382108094
